# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,menaka,15.9182,2.4022,27.76,12,0,5.57,1706375850
1,1,isla vista,34.4133,-119.8610,21.31,52,0,0.00,1706375850
2,2,akureyri,65.6835,-18.0878,-0.04,47,75,6.17,1706375850
3,3,port-aux-francais,-49.3500,70.2167,8.16,91,90,15.00,1706375851
4,4,alaghsas,17.0187,8.0168,23.94,16,0,9.77,1706375851


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 900,
    frame_height = 650
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Date            int64
dtype: object

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] < 35)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
1,1,isla vista,34.4133,-119.8610,21.31,52,0,0.00,1706375850
13,13,hawaiian paradise park,19.5933,-154.9731,21.69,91,0,1.54,1706375853
58,58,lihue,21.9789,-159.3672,22.77,76,20,3.60,1706375866
79,79,suva,-18.1416,178.4415,22.71,94,20,2.06,1706375869
82,82,east london,-33.0153,27.9116,23.52,88,0,2.57,1706375870
111,111,todos santos,23.4500,-110.2167,23.50,47,27,2.00,1706375875
156,156,wailua homesteads,22.0669,-159.3780,22.44,76,20,3.60,1706375884
180,180,alice springs,-23.7000,133.8833,26.79,22,23,2.06,1706375687
190,190,cabo san lucas,22.8909,-109.9124,26.17,46,0,0.00,1706375619
310,310,farafangana,-22.8167,47.8333,24.72,79,8,2.67,1706375930


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
copy_df = ideal_weather_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = copy_df[["City", "Lat", "Lng", "Humidity"]]
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

/var/folders/sm/718jcs250_gclfbkg39pptbw0000gn/T/ipykernel_20692/671350292.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Lat,Lng,Humidity,Hotel Name
1,isla vista,34.4133,-119.8610,52,
13,hawaiian paradise park,19.5933,-154.9731,91,
58,lihue,21.9789,-159.3672,76,
79,suva,-18.1416,178.4415,94,
82,east london,-33.0153,27.9116,88,
111,todos santos,23.4500,-110.2167,47,
156,wailua homesteads,22.0669,-159.3780,76,
180,alice springs,-23.7000,133.8833,22,
190,cabo san lucas,22.8909,-109.9124,46,
310,farafangana,-22.8167,47.8333,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = radius
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
isla vista - nearest hotel: The Club & Guest House
hawaiian paradise park - nearest hotel: Lava Tree Tropic Inn
lihue - nearest hotel: Kauai Palms
suva - nearest hotel: Extension 68
east london - nearest hotel: No hotel found
todos santos - nearest hotel: Hotel Casa Tota
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
alice springs - nearest hotel: Aurora Alice Springs
cabo san lucas - nearest hotel: Comfort Rooms
farafangana - nearest hotel: Le Concombre Masqué
lautoka - nearest hotel: Cathay Hotel
la'ie - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
kapa'a - nearest hotel: Pono Kai Resort
mwingi - nearest hotel: Greenvast Hotel
jolo - nearest hotel: No hotel found
tangalle - nearest hotel: Coco Tangalla
bantogon - nearest hotel: Blue Robin's Place
tamanrasset - nearest hotel: فندق الأمان
utete - nearest hotel: No hotel found
taltal - nearest hotel: Hostería Taltal
birao - nearest hotel: No hotel found
siem reap - 

,City,Lat,Lng,Humidity,Hotel Name
1,isla vista,34.4133,-119.8610,52,The Club & Guest House
13,hawaiian paradise park,19.5933,-154.9731,91,Lava Tree Tropic Inn
58,lihue,21.9789,-159.3672,76,Kauai Palms
79,suva,-18.1416,178.4415,94,Extension 68
82,east london,-33.0153,27.9116,88,No hotel found
111,todos santos,23.4500,-110.2167,47,Hotel Casa Tota
156,wailua homesteads,22.0669,-159.3780,76,Hilton Garden Inn Kauai Wailua Bay
180,alice springs,-23.7000,133.8833,22,Aurora Alice Springs
190,cabo san lucas,22.8909,-109.9124,46,Comfort Rooms
310,farafangana,-22.8167,47.8333,79,Le Concombre Masqué


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 900,
    frame_height = 650,
    hover_cols=["Hotel Name"]
)

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)